In [5]:
import sys
from pathlib import Path

main_path = Path(r'C:\Users\Richard\Desktop\ABCD_Study\Publication\abcd_paper')
sys.path.append(str(main_path))

In [6]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

import src.data.preprocess_data as prep
from src.data.data_loader import RepeatedStratifiedKFoldDataloader
import src.data.var_names as abcd_vars

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
total_df = prep.load_complete_df(main_path / 'data' / 'raw')
total_df.head()

,age,female,married,race_ethnicity_Asian,race_ethnicity_Black,race_ethnicity_Hispanic,race_ethnicity_Other,race_ethnicity_White,high.educ_< HS Diploma,high.educ_Bachelor,...,Major Depressive Disorder,Bipolar Disorder,Psychotic Symptoms,ADHD,Oppositional Defiant Disorder,Conduct Disorder,PTSD,Obsessive Compulsive Disorder,Sleep Problems,Suicide Tendency
src_subject_id,,,,,,,,,,,,,,,,,,,,,
NDAR_INV007W6H7B,126,0,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
NDAR_INV00CY2MDM,130,0,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
NDAR_INV00HEV6HB,124,0,1,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
NDAR_INV00LJVZK2,121,0,0,0,0,0,1,0,1,0,...,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
NDAR_INV00NPMHND,118,1,1,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [8]:
set(['age', 'female']).issubset(total_df.columns)

True

In [9]:
loader = RepeatedStratifiedKFoldDataloader(
    total_df,
    abcd_vars.all_brain_features.features,
    abcd_vars.diagnoses.features,
    abcd_vars.sociodem.features,
    1,
    5,
    0.2
)

In [10]:
train, valid, test, feats = next(iter(loader))

C:\ProgramData\Anaconda3\envs\abcd\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [12]:
len(feats)

254

In [14]:
len(abcd_vars.all_brain_features.features)

258

In [7]:
import xgboost as xgb
import pandas as pd

train_big = pd.concat((train, valid))

In [8]:
dtrain = xgb.DMatrix(data=train_big[abcd_vars.all_brain_features.features], label=train_big['Bipolar Disorder'])
dtrain

In [9]:
params = {
    'objective': 'binary:logistic',
    'booster': 'gbtree',
    'subsample': 0.5,
    'grow_policy': 'depthwise',
    'verbosity': 1,
    'disable_default_eval_metric': 1,
    'blubb': 5757
}

model = xgb.train(params=params, dtrain=dtrain, num_boost_round=1000)

[13:44:57] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { blubb } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [10]:
dtest = xgb.DMatrix(data=test[abcd_vars.all_brain_features.features])
preds = model.predict(dtest)

In [11]:
len(preds)

1657

In [12]:
from sklearn.metrics import roc_auc_score

roc_auc_score(test['Bipolar Disorder'], preds)

0.513035667587634

In [ ]:
xgb_pipe = XGBoostPipeline(x0 = initial_params, )

In [15]:
from skopt.utils import use_named_args
from skopt.space import Dimension, Integer, Real

space = [
    Integer(name='a', low=1, high=11),
    Real(name='b', low=0.01, high=0.02),
    Integer(name='c', low=1111, high=6666)
]

@use_named_args(space)
def fun(a, b, c):
    print(f"a = {a}")
    print(f"b = {b}")
    print(f"c = {c}")

In [10]:
fun('hi', 'joe', 6)

TypeError: wrapper() takes 1 positional argument but 3 were given

In [17]:
d = [3, 0.01564, 339933]
fun(x=d)

a = 3
b = 0.01564
c = 339933


In [23]:
dic = {'a': 3, 'b':5, 'c':111}
'a' in dic.keys()

True

In [21]:
dic2 = {'d': 90, 'e': 22222}
#dic.update(dic2)

In [25]:
dic.pop('a')

3

In [27]:
dic.pop('a')

KeyError: 'a'

In [62]:
from src.models.xgboost_pipeline import DepthwiseXGBPipeline, ErrorFunctions
import time

In [63]:
log_loss = ErrorFunctions.log_loss()
pipe = DepthwiseXGBPipeline(
    y_col = 'Bipolar Disorder',
    model_dir = main_path / 'models',
    error_function = log_loss,
    include_cols = abcd_vars.all_brain_features.features,
    n_calls = 30
)

In [66]:
tic = time.perf_counter()
pipe.fit(data_train = train, data_valid = valid)
toc = time.perf_counter()
print(f"Training took {(toc - tic)/60:0.2f} minutes")

Training took 13.82 minutes


In [67]:
pipe.hparams

{'learning_rate': 0.006146431731654878,
 'alpha': 8.850403281640631e-05,
 'lambda': 0.22199705855060672,
 'objective': 'binary:logistic',
 'booster': 'gbtree',
 'max_depth': 2,
 'subsample': 0.5,
 'grow_policy': 'depthwise',
 'verbosity': 1,
 'disable_default_eval_metric': 1}

In [71]:
y_pred = pipe.predict(test)
roc_auc_score(test['Bipolar Disorder'], y_pred)

0.5154915805391638

In [20]:
abcd_vars.all_brain_features.features

['sri24precentrallgm',
 'sri24precentralrgm',
 'sri24frontalsuplgm',
 'sri24frontalsuprgm',
 'sri24frontalsuporblgm',
 'sri24frontalsuporbrgm',
 'sri24frontalmidlgm',
 'sri24frontalmidrgm',
 'sri24frontalmidorblgm',
 'sri24frontalmidorbrgm',
 'sri24frontalinfoperlgm',
 'sri24frontalinfoperrgm',
 'sri24frontalinftrilgm',
 'sri24frontalinftrirgm',
 'sri24frontalinforblgm',
 'sri24frontalinforbrgm',
 'sri24rolandicoperlgm',
 'sri24rolandicoperrgm',
 'sri24suppmotorarealgm',
 'sri24suppmotorareargm',
 'sri24olfactorylgm',
 'sri24olfactoryrgm',
 'sri24frontalsupmediallgm',
 'sri24frontalsupmedialrgm',
 'sri24frontalmedorblgm',
 'sri24frontalmedorbrgm',
 'sri24rectuslgm',
 'sri24rectusrgm',
 'sri24insulalgm',
 'sri24insulargm',
 'sri24cingulumantlgm',
 'sri24cingulumantrgm',
 'sri24cingulummidlgm',
 'sri24cingulummidrgm',
 'sri24cingulumpostlgm',
 'sri24cingulumpostrgm',
 'sri24hippocampuslgm',
 'sri24hippocampusrgm',
 'sri24parahippocampallgm',
 'sri24parahippocampalrgm',
 'sri24amygdalalgm